# Feedback functions in _NeMo Guardrails_ apps

This notebook demonstrates how to use feedback functions from within rails apps.
The integration in the other direction, monitoring rails apps using trulens, is
shown in the `nemoguardrails_trurails_example.ipynb` notebook.

We feature two examples of how to integrate feedback in rails apps. This
notebook goes over the simpler of the two. The more complex but ultimately more
concise usage of feedback in rails is shown in `nemoguardrails_feedback_action_example.ipynb`.

In [ ]:
# Install NeMo Guardrails if not already installed.
! pip install nemoguardrails

### Setup keys and trulens_eval

In [ ]:
# This notebook uses openai and huggingface providers which need some keys set.
# You can set them here:

from trulens.keys import check_or_set_keys
check_or_set_keys(
    OPENAI_API_KEY="to fill in",
    HUGGINGFACE_API_KEY="to fill in"
)

# Load trulens, reset the database:
from trulens import Tru
tru = Tru()
tru.reset_database()

## Feedback functions setup

Lets consider some feedback functions. We will define two types: a simple
language match that checks whether output of the app is in the same language as
the input. The second is a set of three for evaluating context retrieval.

In [ ]:
from pprint import pprint

from trulens import Feedback
from trulens import Huggingface
from trulens import OpenAI
from trulens.feedback.feedback import rag_triad

# Initialize provider classes
openai = OpenAI()
hugs = Huggingface()

# Note that we do not specify the selectors (where the inputs to the feedback
# functions come from). This is because we will not be using selectors in these examples.
f_language_match = Feedback(hugs.language_match)

fs_triad = rag_triad(provider=openai)

# Overview of the 4 feedback functions defined.
pprint(f_language_match)
pprint(fs_triad)

### Using Feedback functions without selectors

To make feedback functions available to rails apps without selectors, we can use
the `run` method and provide explicit inputs:

In [ ]:
f_language_match.run(text1="Como estas?", text2="I'm doing well, thank you.")

## Rails app setup

The files created below define a configuration of a rails app adapted from
various examples in the NeMo-Guardrails repository. There is nothing unusual
about the app beyond the knowledge base here being the trulens_eval
documentation. This means you should be able to ask the resulting bot questions
regarding trulens instead of the fictional company handbook as was the case in
the originating example.

Note that new additions to output rail flows in the configuration below. These are setup to run our feedback functions but their definition will come in following colang file.

In [ ]:
%%writefile config.yaml
# Adapted from NeMo-Guardrails/nemoguardrails/examples/bots/abc/config.yml
instructions:
  - type: general
    content: |
      Below is a conversation between a user and a bot called the trulens Bot.
      The bot is designed to answer questions about the trulens_eval python library.
      The bot is knowledgeable about python.
      If the bot does not know the answer to a question, it truthfully says it does not know.

sample_conversation: |
  user "Hi there. Can you help me with some questions I have about trulens?"
    express greeting and ask for assistance
  bot express greeting and confirm and offer assistance
    "Hi there! I'm here to help answer any questions you may have about the trulens. What would you like to know?"

models:
  - type: main
    engine: openai
    model: gpt-3.5-turbo-instruct

rails:
  output:
    flows:
      - check language match
      # triad defined seperately so hopefully they can be executed in parallel
      - check rag triad groundedness
      - check rag triad relevance
      - check rag triad qs_relevance

### Output flows with feedback

Next we define output flows that include checks using all 4 feedback functions
we defined above. We will create one custom action for each. We start with
language match and use trulens utilities for the other 3 further in this notebook.

***NOTE: In the second example notebook we use a single generic action instead but
that will require additional setup.***

In [ ]:
from nemoguardrails.actions.actions import action

@action(name="language_match")
async def language_match(text1: str, text2: str):
    # Print out some info for demostration purposes:
    print("Checking language match with:", text1, text2)
    res = f_language_match.run(text1=text1, text2=text2).result
    print(f"Result = {res}")
    return res

#### Action invocation

We can now define output flows that execute the custom actions which in turn
evaluate feedback functions. These are the four "subflow"s in the colang below.

***NOTE: We will create custom actions for the rag triad in a cell further in
this notebook. For now, we get their names and signatures.***

In [ ]:
for f in [f_language_match, *fs_triad.values()]:
    print(f.name, f.sig)

In [ ]:
%%writefileinterpolated config.co
# Adapted from NeMo-Guardrails/tests/test_configs/with_kb_openai_embeddings/config.co
define user ask capabilities
  "What can you do?"
  "What can you help me with?"
  "tell me what you can do"
  "tell me about you"

define bot inform language mismatch
  "I may not be able to answer in your language."

define bot inform triad failure
  "I may may have made a mistake interpreting your question or my knowledge base."

define flow
  user ask trulens
  bot inform trulens

define subflow check language match
  $result = execute language_match(\
    text1=$last_user_message,\
    text2=$bot_message\
  )  
  if $result < 0.8
    bot inform language mismatch
    stop

define subflow check rag triad groundedness
  $result = execute groundedness_measure_with_cot_reasons(\
    source=$relevant_chunks_sep,\
    statement=$bot_message\
  )
  if $result < 0.7
    bot inform triad failure
    stop

define subflow check rag triad relevance
  $result = execute relevance(\
    prompt=$retrieved_for,\
    response=$relevant_chunks_sep\
  )
  if $result < 0.7
    bot inform triad failure
    stop

define subflow check rag triad qs_relevance
  $result = execute qs_relevance(\
    question=$retrieved_for,\
    statement=$bot_message\
  )
  if $result < 0.7
    bot inform triad failure
    stop


## Rails app instantiation

The instantiation of the app does not differ from the steps presented in NeMo.

In [10]:
from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_path(".")
rails = LLMRails(config)

## Register feedback actions with rails app

We need to register each custom action with the rails app. We already created
one above and use a trulens utility to create and regiter the other three for
the rag triad.

In [ ]:
# Register the custom action we created above.
rails.register_action(action=language_match)

# Create custom actions for the rag triad. A utility for creating custom actions
# that do nothing but call a feedback function is provided in trulens
# (FeedbackActions.action_of_feedback). Lets create custom actions for the rag
# triad feedback functions and register them:

from trulens.tru_rails import FeedbackActions
for f in fs_triad.values():
    print(f"registering custom action for feedback function {f.name}")
    # verbose causes the action to print out the inputs it receives when invoked.
    rails.register_action(FeedbackActions.action_of_feedback(f, verbose=True))

## Optional `TruRails` recorder instantiation

Though not required, we can also use a trulens_eval recorder to monitor our app.

In [ ]:
from trulens import TruRails

tru_rails = TruRails(rails)

## Language match test invocation

Lets try to make the app respond in a different language than the question to
try to get the language match flow to abort the output. Note that the verbose
flag in the feedback action we setup in the colang above makes it print out the
inputs and output of the function.

In [ ]:
# This may fail the language match:
with tru_rails as recorder:
    response = rails.generate(messages=[{
        "role": "user",
        "content": "Please answer in Spanish: what does trulens_eval do?"
    }])
    
print(response['content'])

In [ ]:
# Note that the feedbacks involved in the flow are NOT record feedbacks hence
# not available in the usual place:

record = recorder.get()
print(record.feedback_results)

In [ ]:
# This should be ok though sometimes answers in English and the RAG triad may
# fail after language match passes.

with tru_rails as recorder:
    response = rails.generate(messages=[{
        "role": "user",
        "content": "Por favor responda en español: ¿qué hace trulens_eval?"
    }])
    
print(response['content'])

## RAG triad Test

Lets check to make sure all 3 RAG feedback functions will run and hopefully
pass. Note that the "stop" in their flow definitions means that if any one of
them fails, no subsequent ones will be tested.

In [ ]:
# Should invoke retrieval:

with tru_rails as recorder:
    response = rails.generate(messages=[{
        "role": "user",
        "content": "Does trulens support AzureOpenAI as a provider?"
    }])
    
print(response['content'])